# SEACells Versions

### Imports

In [1]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd

In [2]:
import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

from icecream import ic

In [3]:
# from importlib import reload
from SEACells.core import SEACells
# reload(SEACells)

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


### Initialization

In [4]:
num_cells = 1000

In [5]:
ad = sc.read("/home/aparna/DATA/aparnakumar/50000_cells/mouse_marioni_50k.h5ad") 
num_cells = 1000
ad = ad[:num_cells]

In [6]:
def get_data(ad, num_cells, use_gpu, use_sparse): 
  ## User defined parameters

  ## Core parameters 
  # number of SEACells
  n_SEACells = num_cells // 75
  build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

  ## Additional parameters
  n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells

  model = SEACells(ad, 
                                 use_gpu=use_gpu, 
                                 use_sparse=use_sparse, 
                                 build_kernel_on=build_kernel_on, 
                                 n_SEACells=n_SEACells, 
                                 n_waypoint_eigs=n_waypoint_eigs,
                                 convergence_epsilon = 1e-5)
  model.construct_kernel_matrix()
  model.initialize_archetypes()

  start = time.time()
  tracemalloc.start()

  model.fit(min_iter=10, max_iter=150)

  end = time.time()
  tot_time = end - start

  mem = tracemalloc.get_traced_memory()
  tracemalloc.stop()

  assignments = model.get_hard_assignments()
  
  return assignments, tot_time, mem

In [7]:
def all_versions(ad):
    assignments1, time1, mem1 = get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse=False)
    assignments2, time2, mem2 = get_data(ad, num_cells = num_cells, use_gpu = False, use_sparse= True)
    assignments3, time3, mem3 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse=False)
    assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

    # Write the assignments
    assignments = [assignments1, assignments2, assignments3, assignments4] 
    
    # Write the time and memory data
    comparisons = pd.DataFrame({'version': ['v1: no GPU, no sparse', 'v2:  no GPU, yes sparse', 'v3: yes GPU, no sparse', 'v4: yes GPU, yes sparse'], 
                           'time (s)': [time1, time2, time3, time4],
                           'peak memory': [mem1[1], mem2[1], mem3[1], mem4[1]]})
    
    return assignments, comparisons

In [8]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())    

0
0
0


In [9]:
a_cpu = np.ndarray(100, dtype=np.float32)
a = cp.array(a_cpu)
print(a.nbytes)                          # 400
print(mempool.used_bytes())              # 512
print(mempool.total_bytes())             # 512
print(pinned_mempool.n_free_blocks())    # 1

400
512
512
1


In [10]:
B3 = cp.random.random((13, 1000))
print(B3.nbytes)                          
print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())    


104000
104960
104960
1


In [11]:
B3 = cp.sparse.csr_matrix(B3)                        
print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())    

B3 =B3.argmin(axis=0)
print(B3.nbytes)                          
print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())    

157696
369664
1
8000
8704
783872
1


In [12]:
import scipy.sparse

ATA = cp.random.random((1000, 1000))
ATA = cp.sparse.csr_matrix(ATA)


f = ((ATA.multiply(ATA)).sum(axis=0)).ravel()
g = ATA.diagonal().ravel()

k =  13
n = 1000
d = cp.sparse.csr_matrix((k, n), dtype=cp.float64)
omega = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# keep track of selected indices
centers = cp.sparse.csr_matrix((k, n), dtype=cp.float64)

# sampling
for j in tqdm(range(k)):
        score = f / g
        p = score.argmax()
        
        delta_term1 = ATA[:, p].toarray().squeeze()
        #     # print(delta_term1)
        ic(omega[:, p].shape)
        ic(type(omega[:, p].reshape(-1, 1).multiply(omega)))
        delta_term2 = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
        delta = delta_term1 - delta_term2

        #     # some weird rounding errors
        # BOOKMARK
        #delta[p] = [0, delta[p]].max()

        #     o = delta / np.max([np.sqrt(delta[p]), 1e-6])
        #     omega_square_norm = np.linalg.norm(o) ** 2
        #     omega_hadamard = np.multiply(o, o)
        #     term1 = omega_square_norm * omega_hadamard

        #     # update f (term2)
        #     pl = np.zeros(n)
        #     for r in range(j):
        #         omega_r = omega[r, :]
        #         pl += np.dot(omega_r, o) * omega_r

        #     ATAo = (ATA @ o.reshape(-1, 1)).ravel()
        #     term2 = np.multiply(o, ATAo - pl)

        #     # update f
        #     f += -2.0 * term2 + term1

        #     # update g
        #     g += omega_hadamard

        #     # store omega and delta
        #     d[j, :] = delta
        #     omega[j, :] = o

        #     # add index
        #     centers[j] = int(p)

        # return centers

  0%|          | 0/13 [00:00<?, ?it/s]

ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
  8%|▊         | 1/13 [00:00<00:02,  5.31it/s]ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
 31%|███       | 4/13 [00:00<00:00, 14.06it/s]ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| omega[:, p].shape: (13, 1)
ic| type(omega[:, p].reshape(-1, 1).multiply(omega)): <class 'cupyx.scipy.sparse._csr.csr_matrix'

In [13]:
delta_term1.shape

(1000,)

In [14]:
p = cp.array([121])
omega = cp.sparse.csr_matrix((14, 1000), dtype=cp.float64)
thing = omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze()
ic(thing.shape)
ic(type(thing))


type(thing.sum(axis=0))



ic| thing.shape: (1000,)
ic| type(thing): <class 'cupy.ndarray'>


cupy.ndarray

In [15]:
n = 1000
k = 14

# precompute M.T * M
# ATA = M.T @ M
ATA = cupyx.scipy.sparse.csr_matrix((n, n))


f = cp.array((ATA.multiply(ATA)).sum(axis=0)).ravel()
g = cp.array(ATA.diagonal()).ravel()

d = cp.sparse.csr_matrix((k, n))
omega = cp.sparse.csr_matrix((k, n))

# keep track of selected indices
centers = cp.sparse.csr_matrix((k, n))

# sampling
for j in tqdm(range(k)):
    # Compute score, dividing the sparse f by the sparse g
    score = f / g
            
    # Compute p, which is the largest score
    p = cp.argmax(score)

    # Compute delta_term1 to be the column of ATA at index p
    delta_term1 = ATA[:, p].toarray().squeeze()

    # Compute delta_term2 to be the sum of the outer product of omega and itself
    delta_term2 = (omega[:, p].reshape(-1, 1).multiply(omega).sum(axis=0).squeeze())
    delta = delta_term1 - delta_term2

    # some weird rounding errors
    delta[p] = max([0, delta[p]])

    o = delta / max([cp.sqrt(delta[p]), 1e-6])
    omega_square_norm = cp.linalg.norm(o) ** 2
    omega_hadamard = cp.multiply(o, o)
    term1 = omega_square_norm * omega_hadamard

    # update f (term2)
    pl = cp.zeros(n)
    for r in range(j):
        omega_r = omega[r, :]
        pl += omega_r.dot(o) * omega_r

    ATAo = (ATA @ o.reshape(-1, 1)).ravel()
    term2 = o *  (ATAo - pl)

    # update f
    f += -2.0 * term2 + term1

    # update g
    g += omega_hadamard

    # store omega and delta
    d[j, :] = delta
    omega[j, :] = o

    # add index
    centers[j] = int(p)

centers

  0%|          | 0/14 [00:00<?, ?it/s]/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
100%|██████████| 14/14 [00:00<00:00, 71.41it/s]


In [16]:
import cupy

kernel_matrix =  cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)
# Make a sparse csr matrix of random data 
reconstruction = cupyx.scipy.sparse.rand(1000, 1000, density=0.01, format='csr', dtype=np.float64)

ic(type(kernel_matrix))
ic(type(reconstruction))



cp.linalg.norm(kernel_matrix - reconstruction)

ic| type(kernel_matrix): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| type(reconstruction): <class 'cupyx.scipy.sparse._csr.csr_matrix'>


array(17.72535222)

In [17]:
assignments4, time4, mem4 = get_data(ad, num_cells = num_cells, use_gpu = True, use_sparse= True)

SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!
build_graph.SEACellGraph completed
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Building similarity LIL matrix...


...

Starting iteration 30.


/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
ic| '_updateB'
ic| 'step'
ic| 'updateA'


Completed iteration 30.


/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
ic| '_updateB'


Converged after 31 iterations.


In [18]:
assignments, comparisons = all_versions(ad)

NOT SPARSE AND NOT GPUT
TRYING SEACellsCPUDense
Welcome to SEACells!
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Building similarity LIL matrix...


...

Starting iteration 30.


/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
ic| '_updateB'
ic| 'step'
ic| 'updateA'


Completed iteration 30.


/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
ic| '_updateB'


Converged after 31 iterations.


In [19]:
# Display comparisons 
comparisons

,version,time (s),peak memory
0,"v1: no GPU, no sparse",0.992179,24487239
1,"v2: no GPU, yes sparse",32.136723,24604988
2,"v3: yes GPU, no sparse",1.401195,24713684
3,"v4: yes GPU, yes sparse",30.467086,287554


## Comparisons 

In [75]:
## Core parameters 
# number of SEACells
n_SEACells = num_cells // 75
build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

## Additional parameters
n_waypoint_eigs = 10
  
# Initialize
model1 = SEACells(ad, use_gpu=False, use_sparse=False, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)
model4 = SEACells(ad, use_gpu=True, use_sparse=True, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)
model3 = SEACells(ad, use_gpu=True, use_sparse=False, build_kernel_on=build_kernel_on, n_SEACells=n_SEACells, n_waypoint_eigs=n_waypoint_eigs, convergence_epsilon = 1e-5)

NOT SPARSE AND NOT GPUT
TRYING SEACellsCPUDense
Welcome to SEACells!
SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!
NOT SPARSE AND GPU
TRYING SEACellsGPUDense
Welcome to SEACells GPU!


In [76]:
model1.construct_kernel_matrix() 

Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...


  0%|          | 0/1000 [00:00<?, ?it/s]

Building similarity LIL matrix...


  0%|          | 0/1000 [00:00<?, ?it/s]

Constructing CSR matrix...


In [77]:
K= model1.K 
kernel_matrix = model1.kernel_matrix 

In [78]:
import scipy.sparse
model3.K = model1.K
# convert model1.K to the right type (cupyx.scipy.sparse._csr.csr_matrix) 
model4.K = cupyx.scipy.sparse.csr_matrix(model1.K)

ic(type(model1.K))
ic(type(model3.K)) 
ic(type(model4.K)) 

ic(model1.K.shape)
ic(model3.K.shape)
ic(model4.K.shape)

ic| type(model1.K): <class 'scipy.sparse._csr.csr_matrix'>
ic| type(model3.K): <class 'scipy.sparse._csr.csr_matrix'>
ic| type(model4.K): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model1.K.shape: (1000, 1000)


ic| model3.K.shape: (1000, 1000)
ic| model4.K.shape: (1000, 1000)


(1000, 1000)

In [79]:
import scipy

model3.kernel_matrix = model1.kernel_matrix 
model4.kernel_matrix = cupyx.scipy.sparse.csr_matrix(model1.kernel_matrix)


ic(type(model1.kernel_matrix))
ic(type(model3.kernel_matrix)) 
ic(type(model4.kernel_matrix)) 

ic(model1.kernel_matrix.shape)
ic(model3.kernel_matrix.shape)
ic(model4.kernel_matrix.shape)

ic| type(model1.kernel_matrix): <class 'scipy.sparse._csr.csr_matrix'>
ic| type(model3.kernel_matrix): <class 'scipy.sparse._csr.csr_matrix'>
ic| type(model4.kernel_matrix): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model1.kernel_matrix.shape: (1000, 1000)
ic| model3.kernel_matrix.shape: (1000, 1000)
ic| model4.kernel_matrix.shape: (1000, 1000)


(1000, 1000)

In [80]:
model1.initialize_archetypes()

Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...
Done.
Sampling waypoints ...
Done.
Selecting 8 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...


100%|██████████| 15/15 [00:00<00:00, 1687.35it/s]

Selecting 5 cells from greedy initialization.


In [81]:
model3.archetypes = model1.archetypes 
model4.archetypes = cp.array(model1.archetypes)

ic(model1.archetypes.shape)
ic(model3.archetypes.shape)
ic(model4.archetypes.shape)

ic(type(model1.archetypes))
ic(type(model3.archetypes))
ic(type(model4.archetypes))


ic| model1.archetypes.shape: (13,)
ic| model3.archetypes.shape: (13,)
ic| model4.archetypes.shape: (13,)
ic| type(model1.archetypes): <class 'numpy.ndarray'>
ic| type(model3.archetypes): <class 'numpy.ndarray'>
ic| type(model4.archetypes): <class 'cupy.ndarray'>


cupy.ndarray

In [82]:
model1.initialize()
ic(model1.archetypes.shape) 
ic(type(model1.archetypes)) 

ic(model1.k)
ic(type(model1.k))

ic(model1.B0.shape)
ic(type(model1.B0))

ic(model1.A0.shape)
ic(type(model1.A0))

ic(model1.A_.shape)
ic(type(model1.A_)) 

ic(model1.B_.shape)
ic(type(model1.B_))

ic(model1.convergence_threshold)
ic(type(model1.convergence_threshold))

ic| model1.archetypes.shape: (13,)
ic| type(model1.archetypes): <class 'numpy.ndarray'>
ic| model1.k: 13
ic| type(model1.k): <class 'int'>
ic| model1.B0.shape: (1000, 13)
ic| type(model1.B0): <class 'numpy.ndarray'>
ic| model1.A0.shape: (13, 1000)
ic| type(model1.A0): <class 'numpy.ndarray'>
ic| model1.A_.shape: (13, 1000)


ic| type(model1.A_): <class 'numpy.ndarray'>
ic| model1.B_.shape: (1000, 13)
ic| type(model1.B_): <class 'numpy.ndarray'>
ic| model1.convergence_threshold: 0.0005534946234277484
ic| type(model1.convergence_threshold): <class 'numpy.float64'>


Randomly initialized A matrix.
Setting convergence threshold at 0.00055


numpy.float64

In [83]:
model3.initialize()
ic(model3.archetypes.shape)
ic(type(model3.archetypes))

ic(model3.k)
ic(type(model3.k))

ic(model3.B0.shape)
ic(type(model3.B0))

ic(model3.A0.shape)
ic(type(model3.A0))

ic(model3.A_.shape)
ic(type(model3.A_))

ic(model3.B_.shape)
ic(type(model3.B_))

ic(model3.convergence_threshold)
ic(type(model3.convergence_threshold))

ic| model3.archetypes.shape: (13,)
ic| type(model3.archetypes): <class 'numpy.ndarray'>
ic| model3.k: 13
ic| type(model3.k): <class 'int'>
ic| model3.B0.shape: (1000, 13)
ic| type(model3.B0): <class 'numpy.ndarray'>
ic| model3.A0.shape: (13, 1000)
ic| type(model3.A0): <class 'numpy.ndarray'>
ic| model3.A_.shape: (13, 1000)
ic| type(model3.A_): <class 'numpy.ndarray'>
ic| model3.B_.shape: (1000, 13)
ic| type(model3.B_): <class 'numpy.ndarray'>
ic| model3.convergence_threshold: 0.0005534982391880163
ic| type(model3.convergence_threshold): <class 'numpy.float64'>


Randomly initialized A matrix.
Setting convergence threshold at 0.00055


numpy.float64

In [84]:
k = len(model4.archetypes)
cols = cp.arange(k)
rows = model4.archetypes
shape = (n, k) 
data = cp.ones(len(rows))
B0 = cupyx.scipy.sparse.csr_matrix((cp.ones(len(rows)), (rows, cols)), shape=shape)
B = B0.copy()



In [85]:
model4.initialize() 
ic(model4.archetypes.shape)
ic(type(model4.archetypes))

ic(model4.k)
ic(type(model4.k))

ic(model4.B0.shape)
ic(type(model4.B0))

ic(model4.A0.shape)
ic(type(model4.A0))

ic(model4.A_.shape)
ic(type(model4.A_))

ic(model4.B_.shape)
ic(type(model4.B_))

ic(model4.convergence_threshold)
ic(type(model4.convergence_threshold))

ic| 'flag'
ic| 'updateA'
/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '


Randomly initialized A matrix.


ic| model4.archetypes.shape: (13,)
ic| type(model4.archetypes): <class 'cupy.ndarray'>
ic| model4.k: 13
ic| type(model4.k): <class 'int'>
ic| model4.B0.shape: (1000, 13)
ic| type(model4.B0): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model4.A0.shape: (13, 1000)
ic| type(model4.A0): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model4.A_.shape: (13, 1000)
ic| type(model4.A_): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model4.B_.shape: (1000, 13)
ic| type(model4.B_): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| model4.convergence_threshold: array(0.00055353)
ic| type(model4.convergence_threshold): <class 'cupy.ndarray'>


Convergence threshold set to 0.000553529368094502 based on epsilon = 1e-05


cupy.ndarray

In [86]:
# k = model4.k
# cols = cp.arange(k)
# rows = model4.archetypes
# shape = (n, k)
# B0 = cupyx.scipy.sparse.csr_matrix((cp.ones(len(rows)), (rows, cols)), shape=shape)

In [87]:
# archetypes_per_cell = int(k * 0.25)
# rows = np.random.randint(0, k, size=(n, archetypes_per_cell)).reshape(-1)
# columns = np.repeat(np.arange(n), archetypes_per_cell)

# ic(rows.shape)
# ic(columns.shape)
# ic(k, n)

# A0 = scipy.sparse.csr_matrix((np.random.random(len(rows)), (rows, columns)), shape=(k, n))


In [88]:
# archetypes_per_cell = int(k * 0.25)
# rows = np.random.randint(0, k, size=(n, archetypes_per_cell)).reshape(-1)
# columns = np.repeat(np.arange(n), archetypes_per_cell)
#             # print(type(rows))
#             # print(type(columns))

# A0 = scipy.sparse.csr_matrix((np.random.random(len(rows)), (rows, columns)), shape=(k, n))
#             # print(type(A0))
#             # print("STARTING NORMALIZE")
# A0 = cupyx.scipy.sparse.csr_matrix(normalize(A0, axis=0, norm="l1"))


In [89]:
# Now we need to compare 

ic(np.allclose(model1.archetypes, model3.archetypes))
ic(np.allclose(model1.archetypes, model4.archetypes))

ic(np.allclose(model1.k, model3.k))
ic(np.allclose(model1.k, model4.k))

ic(np.allclose(model1.B0, model3.B0))
ic(np.allclose(model1.B0, model4.B0.get().todense()))

ic(np.allclose(model1.A0, model3.A0))
ic(np.allclose(model1.A0, model4.A0.get().todense()))

ic(np.allclose(model1.A_, model3.A_))
ic(np.allclose(model1.A_, model4.A_.get().todense()))

ic(np.allclose(model1.B_, model3.B_))
ic(np.allclose(model1.B_, model4.B_.get().todense()))

ic| np.allclose(model1.archetypes, model3.archetypes): True
ic| np.allclose(model1.archetypes, model4.archetypes): array(True)
ic| np.allclose(model1.k, model3.k): True
ic| np.allclose(model1.k, model4.k): True
ic| np.allclose(model1.B0, model3.B0): True
ic| np.allclose(model1.B0, model4.B0.get().todense()): True
ic| np.allclose(model1.A0, model3.A0): False
ic| np.allclose(model1.A0, model4.A0.get().todense()): False
ic| np.allclose(model1.A_, model3.A_): False
ic| np.allclose(model1.A_, model4.A_.get().todense()): False
ic| np.allclose(model1.B_, model3.B_): True
ic| np.allclose(model1.B_, model4.B_.get().todense()): True


True

In [90]:
# Compare the A's of model 3 and model 4 
ic(np.allclose(model3.A0, model4.A0.get().todense()))
ic(np.allclose(model3.A_, model4.A_.get().todense()))

ic| np.allclose(model3.A0, model4.A0.get().todense()): False
ic| np.allclose(model3.A_, model4.A_.get().todense()): False


False

In [91]:
# Set A0 of model 3 and model 4 to be the same as model 1 
model3.A0 = model1.A0 
model4.A0 = cupyx.scipy.sparse.csr_matrix(cp.array(model1.A0))

# check is close 
ic(np.allclose(model1.A0, model3.A0)) 
ic(np.allclose(model1.A0, model4.A0.get().todense()))

ic| np.allclose(model1.A0, model3.A0): True
ic| np.allclose(model1.A0, model4.A0.get().todense()): True


True

In [92]:
model3.A_ = model3._updateA(model3.B0, model3.A0)
model4.A_ = model4._updateA(model4.B0, model4.A0)

# Check is close 
ic(np.allclose(model1.A_, model3.A_)) 
ic(np.allclose(model1.A_, model4.A_.get().todense()))

ic| 'updateA'


ic| np.allclose(model1.A_, model3.A_): True
ic| np.allclose(model1.A_, model4.A_.get().todense()): True


True

In [93]:
# Double check on B_ 

ic(np.allclose(model1.B_, model3.B_)) 
ic(np.allclose(model1.B_, model4.B_.get().todense()))

ic| np.allclose(model1.B_, model3.B_): True
ic| np.allclose(model1.B_, model4.B_.get().todense()): True


True

In [94]:
RSS3 = model3.compute_RSS(model3.A_, model3.B_)
RSS4 = model4.compute_RSS(model4.A_, model4.B_)
RSS1 = model1.compute_RSS(model1.A_, model1.B_)

ic(type(RSS1)) 
ic(type(RSS3)) 
ic(type(RSS4)) 

ic(RSS1.shape) 
ic(RSS3.shape) 
ic(RSS4.shape)

ic(np.allclose(RSS1, RSS3)) 
ic(np.allclose(RSS1, RSS4))

ic| type(RSS1): <class 'numpy.float64'>
ic| type(RSS3): <class 'numpy.float64'>


ic| type(RSS4): <class 'cupy.ndarray'>
ic| RSS1.shape: ()
ic| RSS3.shape: ()
ic| RSS4.shape: ()
ic| np.allclose(RSS1, RSS3): True
ic| np.allclose(RSS1, RSS4): array(True)


array(True)

In [95]:
import scipy.sparse

reconstruction1 = model1.compute_reconstruction(model1.A_, model1.B_)
reconstruction4 = model4.compute_reconstruction(cupyx.scipy.sparse.csr_matrix(cp.array(model1.A_)), cupyx.scipy.sparse.csr_matrix(cp.array(model1.B_)))
reconstruction3 = model3.compute_reconstruction(model3.A_, model3.B_)

ic(type(reconstruction1)) 
ic(type(reconstruction3))
ic(type(reconstruction4)) 

ic(reconstruction1.shape) 
ic(reconstruction3.shape)
ic(reconstruction4.shape)

ic(np.allclose(reconstruction1, reconstruction3))
ic(np.allclose(reconstruction1, reconstruction4.get().todense()))

ic| type(reconstruction1): <class 'numpy.ndarray'>
ic| type(reconstruction3): <class 'numpy.ndarray'>
ic| type(reconstruction4): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| reconstruction1.shape: (1000, 1000)
ic| reconstruction3.shape: (1000, 1000)


ic| reconstruction4.shape: (1000, 1000)
ic| np.allclose(reconstruction1, reconstruction3): True
ic| np.allclose(reconstruction1, reconstruction4.get().todense()): True


True

In [96]:
import scipy.sparse

diff1 = model1.kernel_matrix - reconstruction1 
diff3 = model3.kernel_matrix - reconstruction3
ic(type(model1.kernel_matrix) )


diff4 = cupyx.scipy.sparse.csr_matrix(model1.kernel_matrix) - reconstruction4 

ic(type(diff1)) 
ic(type(diff3))
ic(type(diff4)) 

ic(diff1.shape) 
ic(diff3.shape )
ic(diff4.shape) 

ic(np.allclose(diff1, diff3))
ic(np.allclose(diff1, diff4.get().todense()))

ic| type(model1.kernel_matrix): <class 'scipy.sparse._csr.csr_matrix'>
ic| type(diff1): <class 'numpy.matrix'>
ic| type(diff3): <class 'numpy.matrix'>
ic| type(diff4): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| diff1.shape: (1000, 1000)
ic| diff3.shape: (1000, 1000)
ic| diff4.shape: (1000, 1000)
ic| np.allclose(diff1, diff3): True


ic| np.allclose(diff1, diff4.get().todense()): True


True

In [97]:
# take the norm of the difference 
import scipy.sparse.linalg
from scipy.sparse.linalg import norm


norm1 = np.linalg.norm(diff1) 
norm3 = np.linalg.norm(diff3)
norm4 = np.linalg.norm(diff4.get().todense())
norm4 = norm(diff4.get())

ic(type(norm1)) 
ic(type(norm3)) 
ic(type(norm4)) 

ic(norm1)
ic(norm3)
ic(norm4)

ic| type(norm1): <class 'numpy.float64'>
ic| type(norm3): <class 'numpy.float64'>
ic| type(norm4): <class 'numpy.float64'>
ic| norm1: 55.34946234277484
ic| norm3: 55.34946234277484
ic| norm4: 55.34946234277484


55.34946234277484

In [98]:
# check A and B 
ic(model1.A_.shape)
ic(model3.A_.shape)
ic(model4.A_.shape)

ic(model1.B_.shape)
ic(model3.B_.shape)
ic(model4.B_.shape)


ic| model1.A_.shape: (13, 1000)
ic| model3.A_.shape: (13, 1000)
ic| model4.A_.shape: (13, 1000)
ic| model1.B_.shape: (1000, 13)
ic| model3.B_.shape: (1000, 13)
ic| model4.B_.shape: (1000, 13)


(1000, 13)

In [101]:
A1 = model1._updateA(model1.B_, model1.A_)
A3 = model3._updateA(model3.B_, model3.A_)
A4 = model4._updateA(model4.B_, model4.A_)

ic| 'updateA'


/home/aparna/miniconda3/lib/python3.9/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '


In [102]:
model1._updateB(A1, model1.B_)
model3._updateB(A3, model3.B_)
model4._updateB(A4, model4.B_)

ic| '_updateB'


In [ ]:
B = model4.B_ 
A_prev = model4.A_ 

n, k = B.shape
# print(n, k)
A = A_prev
# ic(type(A))

t = 0  # current iteration (determine multiplicative update)

# print("bookmark")
Ag = A
# ic(type(A))
Bg = B

ic(type(model4.K))
Kg = model4.K

ic| type(model4.K): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
